In [4]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import pandas as pd
import time
import torch
from torch import nn
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

from torchtext.datasets import YelpReviewFull, YelpReviewPolarity, AmazonReviewPolarity

In [5]:
class TextClassification(nn.Module):
      def __init__(self, vocab_size, embedding_dim, num_classes, hidden_dim):
          super(TextClassification, self).__init__()
          self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim)          
          self.fc1 = nn.Linear(embedding_dim, hidden_dim)
          self.fc2 = nn.Linear(hidden_dim, num_classes)
          self.init_weight()
      def init_weight(self):
          initrange = 0.5
          self.embedding.weight.data.uniform_(-initrange, initrange)
          self.fc1.weight.data.uniform_(-initrange, initrange)
          self.fc1.bias.data.zero_()
          self.fc2.weight.data.uniform_(-initrange, initrange)
          self.fc2.bias.data.zero_()

      def forward(self, text, offset):
          out = self.embedding(text, offset)
          out = torch.relu(self.fc1(out))
          return self.fc2(out)

In [6]:
def get_token(data_iter, tokenizer):
    for _, text in data_iter:
        yield tokenizer(text)

def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]
  for (_label, _text) in batch:
      label_list.append(label_pipeline(_label))
      processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
      text_list.append(processed_text)
      offsets.append(processed_text.size(0))
  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  text_list = torch.cat(text_list)
  return label_list.to(device), text_list.to(device), offsets.to(device)    


def train(dataloader, model, optimizer):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| {:5d}/{:5d} batches | accuracy {:8.3f}'.format(idx, len(dataloader), total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader, model):
    model.eval()
    total_acc, total_count = 0, 0
    df = {}
    pred, org = [], []

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()

            pred.extend(predited_label.argmax(1))
            org.extend(label)
            total_count += label.size(0)
    df = pd.DataFrame({'pred': pred, 'org': org})
    return total_acc/total_count, df

In [7]:
def main(ds_name):
      model = TextClassification(vocab_size, embedding_dim, num_classes, hidden_dim)

      optimizer = torch.optim.SGD(model.parameters(), lr=LR)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
      total_accu = None
      train_iter, test_iter = ds_name()
      train_dataset = to_map_style_dataset(train_iter)
      test_dataset = to_map_style_dataset(test_iter)



      num_train = int(len(train_dataset) * 0.95)
      split_train_, split_valid_ = \
          random_split(train_dataset, [num_train, len(train_dataset) - num_train])

      train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                                    shuffle=True, collate_fn=collate_batch)
      valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                                    shuffle=True, collate_fn=collate_batch)
      test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)

      for epoch in range(1, EPOCHS + 1):
          epoch_start_time = time.time()
          train(train_dataloader, model, optimizer)
          accu_val, df = evaluate(valid_dataloader, model)
          if total_accu is not None and total_accu > accu_val:
            scheduler.step()
          else:
            total_accu = accu_val
          print('-' * 59)
          print('| end of epoch {:3d} | time: {:5.2f}s | valid accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
          print('-' * 59)
      return df


In [8]:
def classwise_prediction_accuracy(df):
    df['pred'] = df['pred'].apply(lambda x: int(x))
    df['org'] = df['org'].apply(lambda x: int(x))
    df['Match'] = df.apply(lambda x: 1 if x['pred']==x['org'] else 0, axis=1)
    df.groupby('org')['Match'].sum()
    return df.groupby('org')['Match'].sum()/df['org'].value_counts().reset_index().sort_values(by='index')['org']

In [9]:
dataset_list = [YelpReviewPolarity, AmazonReviewPolarity]

for ds_name in dataset_list:
    tokenizer = get_tokenizer("basic_english")

    criterion = torch.nn.CrossEntropyLoss()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hyperparameters
    EPOCHS =  20 # epoch
    LR = 5  # learning rate
    BATCH_SIZE = 96 # batch size for training

    train_iter = ds_name(split='train')
    vocab_out = build_vocab_from_iterator(get_token(train_iter, tokenizer), specials=["<unk>"],  min_freq= 5)
    vocab_out.set_default_index(vocab_out['<unk>'])

    text_pipeline = lambda x: vocab_out(tokenizer(x))
    label_pipeline = lambda x: x -1

    train_iter = ds_name(split='train')
    vocab_size = len(vocab_out)
    embedding_dim = 64
    hidden_dim = 32
    num_classes = len(set([label for label, _ in train_iter]))
    df_out = main(ds_name)
    print(f'Classwiswe Prediction for {ds_name} dataset: {classwise_prediction_accuracy(df_out)}')